In [1]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxml

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def extract_names_values(line):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (line.br):
        line.br.extract()
    if line.a:
        line.a.extract()
    if line.sup:
        line.sup.extract()
        
    name_value = ' '.join(line.contents)
    
    # Filter the digit and empty names
    if not(name_value.strip().isdigit()):
        name_value = name_value.strip()
        return name_value    

In [4]:
static_url = "https://www.allamericanfordofhackensack.com/searchused.aspx?Features=3rd+Rear+Seat&Make=Ford&Model=Explorer&Trim=XLT&Year=2021"

In [5]:

html_data = requests.get(static_url).text

In [6]:

soup = BeautifulSoup(html_data, "lxml")

In [7]:
#print(soup.prettify())

In [8]:

data1 = soup.find_all("div", {"class": "col-xs-12 visible-xs margin-top-1x"})

data1[0]

<div class="col-xs-12 visible-xs margin-top-1x"> <pricing-stack><ul class="list-unstyled priceBlock priceBlockResponsiveSrpMobile"><li class="priceBlockItem priceBlockItemPrice"><span class="priceBlockItemPriceLabel">Market Price: </span><span class="pull-right">$35,995</span></li><li class="priceBlockItem priceBlockItemRebates"></li> <li class="priceBlockItem priceBlockItemSpacer" style="margin-top:10px;"></li><li class="priceBlockItem priceBlockItemPrice" style="padding-top:0px;padding-bottom:0px;"><span class="priceBlockItemPriceLabel" style="font-weight:bold;color:#239930;font-size:1em;">All American Discount: </span><span class="pull-right" style="font-weight:bold;color:#239930;font-size:1em;">-$5,000</span></li><li class="priceBlockItem priceBlockItemDivider"><div style="margin:5px 0; border-top:1px solid #ddd;width:100%;"></div></li><li class="priceBlockItem priceBlockItemPrice"><span class="priceBlockItemPriceLabel" style="font-weight:bold;font-size:1.2em;">Internet Price </spa

In [9]:
data2 = soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"})

print(data2[19])

<ul class="list-unstyled priceBlock priceBlockResponsiveSrpMobile"><li class="priceBlockItem priceBlockItemPrice"><span class="priceBlockItemPriceLabel">Market Price: </span><span class="pull-right">Call For Price</span></li><li class="priceBlockItem priceBlockItemRebates"></li> <li class="priceBlockItem priceBlockItemSpacer" style="margin-top:10px;"></li><li class="priceBlockItem priceBlockItemSpacer" style="margin-top:10px;"></li></ul>


In [10]:
for ultag in soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"}, limit = 2):
     for litag in ultag.find_all('li'):
         print(litag.text)

Market Price: $35,995


All American Discount: -$5,000

Internet Price $30,995
Dealer Doc Fee: +$699

Market Price: $35,995


All American Discount: -$2,000

Internet Price $33,995
Dealer Doc Fee: +$699



In [11]:
# 
for ultag in soup.find_all("ul", {"class": "list-unstyled srpVehicleDetails hidden-xs"}, limit = 2):
     for litag in ultag.find_all('li'):
             print(litag.text)

 Body Style: Sport Utility
 Model Code: K8D
 Engine: 2.3L I4 16V GDI DOHC Turbo
 Drive Type: 4WD
 Transmission: Automatic
 Ext. Color: White
 Mileage: 27,796
 VIN #: 1FMSK8DH8MGB66051 
 Stock #: HR1018A
  Status: Available
  ndow Sticker
 Body Style: Sport Utility
 Model Code: K8D
 Engine: 2.3L I4 16V GDI DOHC Turbo
 Drive Type: 4WD
 Transmission: Automatic
 Ext. Color: Gray
 Mileage: 15,910
 VIN #: 1FMSK8DH8MGA10401 
 Stock #: 23T1000A
  Status: Available
  ndow Sticker


In [12]:

column_names = []

for line in data1[0].find_all("strong"):
    column_name = extract_names_values(line)
    
    if (column_name != None) and (len(column_name) > 0):
        
        column_names.append(column_name)

Check the extracted column names


In [13]:
print(column_names)

['Body Style:', 'Model Code:', 'Engine:', 'Drive Type:', 'Transmission:', 'Ext. Color:', 'Mileage:', 'VIN #:', 'Stock #:', 'Status:']


In [14]:

column_names2 = []

for line in data2[0].find_all("span", {"class": "priceBlockItemPriceLabel"}):
    column_name = extract_names_values(line)
    
    if (column_name != None) and (len(column_name) > 0):
        
        column_names2.append(column_name)

In [15]:
print(column_names2)

['Market Price:', 'All American Discount:', 'Internet Price', 'Dealer Doc Fee:']


In [16]:
import re
 
def remove_char(list, char):
    return [re.sub(char, '', i) for i in list]

# initialize character
char = ':'

column_names = remove_char(column_names, char)
column_names2 = remove_char(column_names2,char)

column_names = column_names + column_names2

print(column_names)

['Body Style', 'Model Code', 'Engine', 'Drive Type', 'Transmission', 'Ext. Color', 'Mileage', 'VIN #', 'Stock #', 'Status', 'Market Price', 'All American Discount', 'Internet Price', 'Dealer Doc Fee']


In [17]:
car_dict= dict.fromkeys(column_names)

print(car_dict)

{'Body Style': None, 'Model Code': None, 'Engine': None, 'Drive Type': None, 'Transmission': None, 'Ext. Color': None, 'Mileage': None, 'VIN #': None, 'Stock #': None, 'Status': None, 'Market Price': None, 'All American Discount': None, 'Internet Price': None, 'Dealer Doc Fee': None}


In [23]:
body_styles = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    style_li = ul_tag.find('li', class_='bodyStyleDisplay')
    if style_li:  # If 'bodyStyleDisplay' exists within this 'ul' tag
        strong_tag = style_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Body Style:':  # Check if it's the 'Body Style:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            body_style = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no body style information found
            body_styles.append(body_style)
    else:
        body_styles.append('Missing')  # If 'bodyStyleDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(body_styles))        
print(body_styles)

model_codes = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    model_li = ul_tag.find('li', class_='modelCodeDisplay')
    if model_li:  # If 'modelCodeDisplay' exists within this 'ul' tag
        strong_tag = model_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Model Code:':  # Check if it's the 'Model Code:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            model_code = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no model information found
            model_codes.append(model_code)
    else:
        model_codes.append('Missing')  # If 'modelCodeDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(model_codes))        
print(model_codes)

engines = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    engine_li = ul_tag.find('li', class_='engineDisplay')
    if engine_li:  # If 'engineDisplay' exists within this 'ul' tag
        strong_tag = engine_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Engine:':  # Check if it's the 'Engine:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            engine = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no engine information found
            engines.append(engine)
    else:
        engines.append('Missing')  # If 'engineDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(engines))        
print(engines)

drive_types = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    drive_li = ul_tag.find('li', class_='driveTrainDisplay')
    if drive_li:  # If 'driveTrainDisplay' exists within this 'ul' tag
        strong_tag = drive_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Drive Type:':  # Check if it's the 'Drive Type:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            drive_type = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no drive type information found
            drive_types.append(drive_type)
    else:
        drive_types.append('Missing')  # If 'driveTrainDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(drive_types))        
print(drive_types)

transmissions = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    transmission_li = ul_tag.find('li', class_='transmissionDisplay')
    if transmission_li:  # If 'transmissionDisplay' exists within this 'ul' tag
        strong_tag = transmission_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Transmission:':  # Check if it's the 'Transmission:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            transmission = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no transmission information found
            transmissions.append(transmission)
    else:
        transmissions.append('Missing')  # If 'transmissionDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(transmissions))        
print(transmissions)

colors = []
for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    ext_color_li = ul_tag.find('li', class_='extColor')
    if ext_color_li:  # If 'extColor' exists within this 'ul' tag
        strong_tag = ext_color_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Ext. Color:':  # Check if it's the 'Ext. Color:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            color = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no color information found
            colors.append(color)
    else:
        colors.append('Missing')  # If 'extColor' does not exist within this 'ul' tag, append 'Missing'
        
print(len(colors))        
print(colors)

mileages = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    mileage_li = ul_tag.find('li', class_='mileageDisplay')
    if mileage_li:  # If 'mileageDisplay' exists within this 'ul' tag
        strong_tag = mileage_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Mileage:':  # Check if it's the 'Mileage:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            mileage = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no mileage information found
            mileages.append(mileage)
    else:
        mileages.append('Missing')  # If 'mileageDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(mileages))        
print(mileages)

vins = []  # Initialize the list to store VINs

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    vin_li = ul_tag.find('li', class_='vinDisplay')
    if vin_li:  # If 'vinDisplay' exists within this 'ul' tag
        strong_tag = vin_li.find('strong')  # Find the <strong> tag
        span_tag = vin_li.find('span')  # Find the <span> tag
        if strong_tag and span_tag:  
            vin = span_tag.text.strip()  # Extract the text from the <span> tag
            vins.append(vin)
        elif strong_tag:  # If only the <strong> tag exists
            next_elem = strong_tag.parent.find_next_sibling()  # Get the next sibling element after the parent node of the <strong> tag
            if next_elem is not None and next_elem.name == 'span':  # Check if the next sibling element is a <span> tag
                vin = next_elem.text.strip()  # Extract the text from the <span> tag
                vins.append(vin)
    else:
        vins.append('Missing')  # If 'vinDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(vins))        
print(vins)

stock_nos = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    stock_li = ul_tag.find('li', class_='stockDisplay')
    if stock_li:  # If 'stockDisplay' exists within this 'ul' tag
        strong_tag = stock_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Stock #:':  # Check if it's the 'Stock #:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            stock_no = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no stock # information found
            stock_nos.append(stock_no)
    else:
        stock_nos.append('Missing')  # If 'stockDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(stock_nos))        
print(stock_nos)

availability = []

for ul_tag in soup.find_all('ul', class_='list-unstyled srpVehicleDetails'):
    available_li = ul_tag.find('li', class_='vehicleStageDisplay')
    if available_li:  # If 'vehicleStageDisplay' exists within this 'ul' tag
        strong_tag = available_li.find('strong')  # Find the <strong> tag
        if strong_tag and strong_tag.text.strip() == 'Status:':  # Check if it's the 'Status:' tag
            next_elem = strong_tag.next_sibling  # Get the next sibling element (which contains the desired text)
            available = next_elem.strip() if next_elem else 'Missing'  # Use 'Missing' if no status information found
            availability.append(available)
    else:
        availability.append('Missing')  # If 'vehicleStageDisplay' does not exist within this 'ul' tag, append 'Missing'

print(len(availability))        
print(availability)


20
['Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility']
20
['K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D']
20
['2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo'

In [24]:
market_prices = []
for ultag in soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"}):
    for litag in ultag.find_all('li'):
        if "Market Price:" in litag.text:
            market_price = float(litag.text.split()[2].replace("$", "").replace(",", "").replace("Call", '0')) # Replace commas with empty strings and convert to float
            market_prices.append(market_price)
            
print(len(market_prices)) 
print(market_prices) # Output: [35995.0, 35995.0]
print()

discounts = []

for ultag in soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"}):
    discount_found = False
    for litag in ultag.find_all('li'):
        if "All American Discount: " in litag.text:
            discount = float(litag.text.split()[3].replace("$", "").replace(",", "").replace("-", ""))
            discounts.append(discount)
            discount_found = True
            break  # Stop searching for discounts in this 'ul' once one is found
    if not discount_found:
        discounts.append(0)  # If no discount is found, append zero       

print(len(discounts))
print(discounts) # Output: [35995.0, 35995.0]
print()

internet_prices = []

for ultag in soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"}):
    internet_price_found = False
    for litag in ultag.find_all('li'):
        if "Internet Price" in litag.text:
            internet_price = float(litag.text.split()[2].replace("$", "").replace(",", ""))
            internet_prices.append(internet_price)
            internet_price_found = True
            break  # Stop searching for discounts in this 'ul' once one is found
    if not internet_price_found:
        internet_prices.append(0)  # If no discount is found, append zero       
            
print(len(internet_prices))
print(internet_prices) # Output: [35995.0, 35995.0]
print()

dealer_fees = []

for ultag in soup.find_all("ul", {"class": "list-unstyled priceBlock priceBlockResponsiveSrpMobile"}):
    dealer_fee_found = False
    for litag in ultag.find_all('li'):
        if "Dealer Doc Fee: " in litag.text:
            dealer_fee = float(litag.text.split()[3].replace("$", "").replace(",", "").replace("+", ""))
            dealer_fees.append(dealer_fee)
            dealer_fee_found = True
            break  # Stop searching for discounts in this 'ul' once one is found
    if not dealer_fee_found:
        dealer_fees.append(0)  # If no discount is found, append zero 

print(len(dealer_fees))
print(dealer_fees) # Output: [35995.0, 35995.0]
print()

20
[35995.0, 35995.0, 35995.0, 35995.0, 34995.0, 35995.0, 33995.0, 32995.0, 0.0, 0.0, 36995.0, 34995.0, 35995.0, 33995.0, 32995.0, 32995.0, 34995.0, 35995.0, 0.0, 0.0]

20
[5000.0, 2000.0, 5000.0, 3300.0, 2801.0, 3500.0, 2000.0, 2000.0, 0, 0, 5300.0, 6000.0, 6000.0, 5000.0, 4000.0, 4000.0, 3500.0, 2000.0, 0, 0]

20
[30995.0, 33995.0, 30995.0, 32695.0, 32194.0, 32495.0, 31995.0, 30995.0, 0, 0, 31695.0, 28995.0, 29995.0, 28995.0, 28995.0, 28995.0, 31495.0, 33995.0, 0, 0]

20
[699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 0, 0, 699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 699.0, 0, 0]



In [25]:

car_dict['Body Style'] = body_styles
car_dict['Model Code'] = model_codes
car_dict['Engine'] = engines
car_dict['Drive Type'] = drive_types
car_dict['Transmission'] = transmissions
car_dict['Ext. Color'] = colors
car_dict['Mileage'] = mileages
car_dict['VIN #'] = vins
car_dict['Stock #'] = stock_nos
car_dict['Status'] = availability
car_dict['Market Price'] = market_prices
car_dict['All American Discount'] = discounts
car_dict['Internet Price'] = internet_prices
car_dict['Dealer Doc Fee'] = dealer_fees

In [26]:
print(car_dict)

{'Body Style': ['Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility', 'Sport Utility'], 'Model Code': ['K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D', 'K8D'], 'Engine': ['2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turbo', '2.3L I4 16V GDI DOHC Turb

In [27]:
car_df = pd.DataFrame(car_dict)

car_df

,Body Style,Model Code,Engine,Drive Type,Transmission,Ext. Color,Mileage,VIN #,Stock #,Status,Market Price,All American Discount,Internet Price,Dealer Doc Fee
0,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,White,"27,796",1FMSK8DH8MGB66051,HR1018A,Available,35995.0,5000.0,30995.0,699.0
1,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Gray,"15,910",1FMSK8DH8MGA10401,23T1000A,Available,35995.0,2000.0,33995.0,699.0
2,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Missing,"27,119",1FMSK8DH5MGA00764,23T1036A,Available,35995.0,5000.0,30995.0,699.0
3,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Missing,"24,889",1FMSK8DH1MGB92006,23T1133A,Available,35995.0,3300.0,32695.0,699.0
4,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,White,"20,053",1FMSK8DH5MGA26023,23T1031A,Available,34995.0,2801.0,32194.0,699.0
5,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Missing,"15,991",1FMSK8DH6MGA54297,23T409A,Available,35995.0,3500.0,32495.0,699.0
6,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Missing,"22,615",1FMSK8DH1MGA75932,23T792A,Available,33995.0,2000.0,31995.0,699.0
7,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Silver,"35,481",1FMSK8DH9MGA32245,24T064A,Available,32995.0,2000.0,30995.0,699.0
8,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Gray,"11,297",1FMSK8DH5MGA66523,P5693A,Available,0.0,0.0,0.0,0.0
9,Sport Utility,K8D,2.3L I4 16V GDI DOHC Turbo,4WD,Automatic,Blue,"29,691",1FMSK8DH3MGA48313,HR1072A,Available,0.0,0.0,0.0,0.0


In [28]:
car_df.describe()

,Market Price,All American Discount,Internet Price,Dealer Doc Fee
count,20.000000,20.000000,20.000000,20.000000
mean,28046.000000,3070.050000,24975.950000,559.200000
std,14432.835517,2023.824688,12899.405215,286.863547
min,0.000000,0.000000,0.000000,0.000000
25%,32995.000000,2000.000000,28995.000000,699.000000
50%,34995.000000,3400.000000,30995.000000,699.000000
75%,35995.000000,5000.000000,32044.750000,699.000000
max,36995.000000,6000.000000,33995.000000,699.000000


In [ ]:
#df.to_csv('ford_explorer_xlt21.csv', index=False)


## Author

Alin Airinei